## DBSCAN

In [ ]:
# Apply DBSCAN to ensure inliers are contiguous
def apply_connectivity_check(inliers, eps=0.5, min_samples=5):
    """
    Apply DBSCAN to ensure the inliers form a connected region.
    
    Parameters:
        inliers (ndarray): Nx3 array of inlier point coordinates.
        eps (float): Maximum distance between points for them to be considered in the same neighborhood.
        min_samples (int): Minimum number of points to form a cluster.
    
    Returns:
        largest_cluster (ndarray): The largest contiguous cluster of points.
    """
    if len(inliers) == 0:
        return np.array([])  # Return empty if no inliers

    db = DBSCAN(eps=eps, min_samples=min_samples).fit(inliers)
    labels = db.labels_

    # Find the largest cluster (most common label excluding noise)
    unique_labels, counts = np.unique(labels[labels >= 0], return_counts=True)
    if len(unique_labels) == 0:
        return np.array([])  # No valid clusters found
    
    largest_cluster_label = unique_labels[np.argmax(counts)]
    largest_cluster = inliers[labels == largest_cluster_label]

    return largest_cluster

## Statistics

In [3]:
def cylinder_statistics(inliers, cylinder_params):
    stats = {}
    
    # Unpack cylinder parameters
    axis_direction, cylinder_center, cylinder_radius = cylinder_params

    # Calculate distances of inliers to the cylinder axis
    vector_to_points = inliers - cylinder_center
    projection_length = np.dot(vector_to_points, axis_direction)
    projection = projection_length[:, np.newaxis] * axis_direction
    perpendicular_distances = np.linalg.norm(vector_to_points - projection, axis=1)

    # Calculate the height of the cylinder
    height = projection_length.max() - projection_length.min()
    stats['height'] = height
    stats['radius'] = cylinder_radius

    # (a) Number of inliers
    stats['num_inliers'] = len(inliers)

    # (b) Absolute mean distance and standard deviation
    stats['mean_distance'] = np.mean(perpendicular_distances)
    stats['std_distance'] = np.std(perpendicular_distances)

    # Separate inliers within and outside the radius
    inliers_within_radius = inliers[perpendicular_distances <= cylinder_radius]
    inliers_outside_radius = inliers[perpendicular_distances > cylinder_radius]

    # (c) Number, average, and standard deviation of inliers within the radius
    stats['num_inliers_within_radius'] = len(inliers_within_radius)
    stats['mean_distance_within_radius'] = np.mean(perpendicular_distances[perpendicular_distances <= cylinder_radius]) if len(inliers_within_radius) > 0 else 0
    stats['std_distance_within_radius'] = np.std(perpendicular_distances[perpendicular_distances <= cylinder_radius]) if len(inliers_within_radius) > 0 else 0

    # (d) Number, average, and standard deviation of inliers outside the radius
    stats['num_inliers_outside_radius'] = len(inliers_outside_radius)
    stats['mean_distance_outside_radius'] = np.mean(perpendicular_distances[perpendicular_distances > cylinder_radius]) if len(inliers_outside_radius) > 0 else 0
    stats['std_distance_outside_radius'] = np.std(perpendicular_distances[perpendicular_distances > cylinder_radius]) if len(inliers_outside_radius) > 0 else 0

    # Centroids of inliers within and outside radius
    stats['centroid_within_radius'] = np.mean(inliers_within_radius, axis=0) if len(inliers_within_radius) > 0 else None
    stats['centroid_outside_radius'] = np.mean(inliers_outside_radius, axis=0) if len(inliers_outside_radius) > 0 else None

    return stats

In [4]:
def plane_statistics(inliers, plane_params):
    plane_info = {}
    a, b, c, d = plane_params
    plane_normal = np.array([a, b, c])

    # (1) A point on the plane (we use the centroid of the inliers here)
    point_on_plane = np.mean(inliers, axis=0)
    plane_info['point_on_plane'] = point_on_plane

    # (2) Two orthogonal vectors on the plane
    if np.allclose(plane_normal, [0, 0, 1]):
        orthogonal_vector1 = np.array([1, 0, 0])
    else:
        orthogonal_vector1 = np.cross(plane_normal, [0, 0, 1])
    orthogonal_vector1 /= np.linalg.norm(orthogonal_vector1)
    orthogonal_vector2 = np.cross(plane_normal, orthogonal_vector1)
    orthogonal_vector2 /= np.linalg.norm(orthogonal_vector2)

    plane_info['orthogonal_vector1'] = orthogonal_vector1
    plane_info['orthogonal_vector2'] = orthogonal_vector2

    # (3) Calculate distances of inliers to the plane
    distances = np.dot(inliers - point_on_plane, plane_normal) / np.linalg.norm(plane_normal)
    
    # (4) Separate inliers by side of the plane
    inliers_positive_side = inliers[distances > 0]
    inliers_negative_side = inliers[distances < 0]

    # General inlier statistics
    plane_info['num_inliers'] = len(inliers)
    plane_info['mean_distance'] = np.mean(np.abs(distances))
    plane_info['std_distance'] = np.std(np.abs(distances))

    # Statistics for inliers on the positive side of the plane
    if len(inliers_positive_side) > 0:
        plane_info['num_inliers_positive'] = len(inliers_positive_side)
        plane_info['mean_distance_positive'] = np.mean(distances[distances > 0])
        plane_info['std_distance_positive'] = np.std(distances[distances > 0])
        plane_info['centroid_positive'] = np.mean(inliers_positive_side, axis=0)
    else:
        plane_info['num_inliers_positive'] = 0
        plane_info['mean_distance_positive'] = None
        plane_info['std_distance_positive'] = None
        plane_info['centroid_positive'] = None

    # Statistics for inliers on the negative side of the plane
    if len(inliers_negative_side) > 0:
        plane_info['num_inliers_negative'] = len(inliers_negative_side)
        plane_info['mean_distance_negative'] = np.mean(distances[distances < 0])
        plane_info['std_distance_negative'] = np.std(distances[distances < 0])
        plane_info['centroid_negative'] = np.mean(inliers_negative_side, axis=0)
    else:
        plane_info['num_inliers_negative'] = 0
        plane_info['mean_distance_negative'] = None
        plane_info['std_distance_negative'] = None
        plane_info['centroid_negative'] = None

    return plane_info

## Mesh

In [ ]:
def create_oriented_cylinder_mesh(cylinder_params, inlier_points):
    axis_direction, cylinder_center, radius = cylinder_params
    axis_direction = axis_direction / np.linalg.norm(axis_direction)  # Ensure unit direction

    # Project inliers onto the axis to determine the height range
    projections = np.dot(inlier_points - cylinder_center, axis_direction)
    min_proj, max_proj = projections.min(), projections.max()
    height = max_proj - min_proj  # Constrained height based on inliers

    # Generate mesh for the cylinder within calculated bounds
    cylinder_mesh = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=height, resolution=20, split=4)
    cylinder_mesh.compute_vertex_normals()

    # Translate to match the inlier centroid position
    mesh_center = cylinder_center + axis_direction * (min_proj + max_proj) / 2
    cylinder_mesh.translate(mesh_center)

    # Align cylinder mesh to the calculated axis direction
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(z_axis, axis_direction)
    rotation_angle = np.arccos(np.clip(np.dot(z_axis, axis_direction), -1.0, 1.0))  # Clamp for numerical stability
    if np.linalg.norm(rotation_axis) > 1e-6:  # Avoid division by zero for nearly aligned vectors
        rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
        rotation_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle)
        cylinder_mesh.rotate(rotation_matrix, center=cylinder_mesh.get_center())
    
    return cylinder_mesh

In [ ]:
def create_oriented_plane_mesh(plane_params, inlier_points):
    a, b, c, d = plane_params
    normal = np.array([a, b, c])

    # Project inlier points onto the plane
    projected_points = inlier_points - (np.dot(inlier_points, normal) + d).reshape(-1, 1) * normal / np.dot(normal, normal)

    # Perform PCA to find the principal directions on the plane
    mean = np.mean(projected_points, axis=0)
    centered_points = projected_points - mean
    _, _, vh = np.linalg.svd(centered_points)

    # The first two right singular vectors are the principal directions
    u = vh[0]
    v = vh[1]

    # Calculate the extent of the points along the principal directions
    u_coords = np.dot(centered_points, u)
    v_coords = np.dot(centered_points, v)
    u_range = np.max(u_coords) - np.min(u_coords)
    v_range = np.max(v_coords) - np.min(v_coords)

    # Create a grid of points on the plane
    resolution = 20
    u_grid = np.linspace(-u_range/2, u_range/2, resolution)
    v_grid = np.linspace(-v_range/2, v_range/2, resolution)
    U, V = np.meshgrid(u_grid, v_grid)

    # Calculate the 3D coordinates of the grid points
    grid_points = mean + U.reshape(-1, 1) * u + V.reshape(-1, 1) * v

    # Create vertices and triangles
    vertices = grid_points
    triangles = []
    for i in range(resolution - 1):
        for j in range(resolution - 1):
            v0 = i * resolution + j
            v1 = v0 + 1
            v2 = (i + 1) * resolution + j
            v3 = v2 + 1
            triangles.extend([[v0, v2, v1], [v1, v2, v3]])

    # Create Open3D mesh
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    mesh.compute_vertex_normals()

    return mesh

In [ ]:
def create_meshes_from_segments(segmented_objects, original_cloud):
    """
    Generate and visualize meshes for each segmented object with color based on model type,
    along with the original point cloud.

    Parameters:
        segmented_objects (list): List of segmented objects from `ransac_main`, containing
                                  model type ('cylinder' or 'plane'), parameters, and inliers.
        original_cloud (np.ndarray): Original point cloud to display alongside the meshes.

    Returns:
        meshes (list): List of colored meshes for each segmented object and the original point cloud.
    """
    meshes = []

    # Convert the original point cloud to Open3D format and add it to the display list
    meshes.append(original_cloud)
    
    for segment in segmented_objects:
        model_type = segment['model']
        params = segment['params']
        inlier_points = segment['inliers']
        
        # Generate the appropriate mesh based on the model type
        if model_type == 'cylinder':
            mesh = create_oriented_cylinder_mesh(params, inlier_points)
            mesh.paint_uniform_color([0, 0, 1])  # Blue color for cylinders
        elif model_type == 'plane':
            mesh = create_oriented_plane_mesh(params, inlier_points)
            mesh.paint_uniform_color([1, 0, 0])  # Red color for planes
        else:
            continue  # Skip any other model types if present
        
        meshes.append(mesh)
    
    # Visualize all generated meshes along with the original cloud
    o3d.visualization.draw_geometries(meshes)
    
    return meshes

## Save

In [ ]:
def save_segments_to_excel(segments, file_path):
    """
    Save segment data to an Excel file, appending if the file exists.
    
    Parameters:
        segments (list of dict): A list where each entry is a dictionary containing details of a segment.
        file_path (str): Path to the Excel file where data should be saved.
    """
    # Prepare an empty list to hold data rows
    data_rows = []

    # Iterate over each segment to extract and organize data
    for segment in segments:
        # Initialize row with common fields
        row = {
            'Model': segment.get('model'),
            'Num Inliers': segment.get('num_inliers', None),
            'Mean Distance': segment.get('mean_distance', None),
            'Std Deviation Distance': segment.get('std_distance', None),
        }
        
        # Add specific fields depending on the model type
        if segment['model'] == 'cylinder':
            row.update({
                'Cylinder Axis': segment['params'][0],
                'Cylinder Center': segment['params'][1],
                'Cylinder Radius': segment['params'][2],
                'Cylinder Height': segment.get('height', None),
                'Num Inliers within Radius': segment.get('num_inliers_within_radius', None),
                'Mean Distance within Radius': segment.get('mean_distance_within_radius', None),
                'Std Deviation within Radius': segment.get('std_distance_within_radius', None),
                'Num Inliers outside Radius': segment.get('num_inliers_outside_radius', None),
                'Mean Distance outside Radius': segment.get('mean_distance_outside_radius', None),
                'Std Deviation outside Radius': segment.get('std_distance_outside_radius', None),
                'Centroid within Radius': segment.get('centroid_within_radius', None),
                'Centroid outside Radius': segment.get('centroid_outside_radius', None)
            })
        elif segment['model'] == 'plane':
            row.update({
                'Plane Normal': segment['params'][:3],  # Normal vector
                'Plane Distance': segment['params'][3],  # Distance from origin (d in ax + by + cz + d = 0)
                'Point on Plane': segment.get('point_on_plane'),
                'Orthogonal Vector 1': segment.get('orthogonal_vector1'),
                'Orthogonal Vector 2': segment.get('orthogonal_vector2'),
                'Num Inliers Positive': segment.get('num_inliers_positive', None),
                'Mean Distance Positive': segment.get('mean_distance_positive', None),
                'Std Deviation Positive': segment.get('std_distance_positive', None),
                'Centroid Positive': segment.get('centroid_positive', None),
                'Num Inliers Negative': segment.get('num_inliers_negative', None),
                'Mean Distance Negative': segment.get('mean_distance_negative', None),
                'Std Deviation Negative': segment.get('std_distance_negative', None),
                'Centroid Negative': segment.get('centroid_negative', None)
            })

        # Ask the user to enter a label for the Name column
        row['Name'] = input(f"Enter a name/label for the {segment['model']} with num_inliers={segment.get('num_inliers', 'N/A')}: ")

        # Append the row to the data list
        data_rows.append(row)

    # Convert to DataFrame
    new_data = pd.DataFrame(data_rows)

    # If the file already exists, read the existing data and append to it
    if os.path.exists(file_path):
        existing_data = pd.read_excel(file_path)
        combined_data = pd.concat([existing_data, new_data], ignore_index=True)
    else:
        # If the file does not exist, just use the new data
        combined_data = new_data

    # Save combined data back to the Excel file
    combined_data.to_excel(file_path, index=False)
    print(f"Data saved successfully to {file_path}")

## Visualize

In [6]:
def visualize_segments_by_model_type(segmented_objects, surface_cloud):
    """
    Visualizes the segmented objects with specific colors for planes and cylinders.
    
    Parameters:
        segmented_objects (list): List of segmented objects with 'model', 'params', and 'inliers'.
        surface_cloud (o3d.geometry.PointCloud): Original surface point cloud.
    """
    geometries = [surface_cloud]
    
    # Define colors for each model type
    color_map = {
        'plane': [1, 0, 0],  # Red for planes
        'cylinder': [0, 0, 1]  # Blue for cylinders
    }
    surface_cloud.paint_uniform_color([0,0,0])
    
    # Loop through each segmented object and assign the corresponding color based on model type
    for segment in segmented_objects:
        model_type = segment['model']
        segment_color = color_map.get(model_type, [0.5, 0.5, 0.5])  # Default grey if model type is unknown
        
        # Create point cloud for the current segment
        segment_cloud = o3d.geometry.PointCloud()
        segment_cloud.points = o3d.utility.Vector3dVector(segment['inliers'])
        segment_cloud.paint_uniform_color(segment_color)
        
        # Append the segment cloud to the geometries list
        geometries.append(segment_cloud)
    
    # Visualize all segments and the surface cloud
    o3d.visualization.draw_geometries(geometries, point_show_normal=True)

In [7]:
def visualize_segments_in_order(segmented_objects):
    """
    Visualizes each segmented object with a unique color in sequence and prints the color name assigned to each object.

    Parameters:
        segmented_objects (list): List of segmented objects with 'model', 'params', and 'inliers'.
    """
    geometries = []

    # Define a palette of colors with names in a specific order
    color_palette = [
        ("Red", [1, 0, 0]),
        ("Green", [0, 1, 0]),
        ("Blue", [0, 0, 1]),
        ("Yellow", [1, 1, 0]),
        ("Pink", [1, 0, 1]),
        ("Cyan", [0, 1, 1]),
        ("Grey", [0.5, 0.5, 0.5]),
        ("Orange", [1, 0.5, 0]),
        ("Purple", [0.5, 0, 0.5]),
        ("Teal", [0, 0.5, 0.5])
    ]

    # Loop through each segmented object and assign colors in sequence
    for i, segment in enumerate(segmented_objects):
        color_name, segment_color = color_palette[i % len(color_palette)]
        
        # Print out the model type, index, and assigned color name
        print(f"Segment {i}: Model = {segment['model']}, Color = {color_name}")

        # Create point cloud for the current segment
        segment_cloud = o3d.geometry.PointCloud()
        segment_cloud.points = o3d.utility.Vector3dVector(segment['inliers'])
        segment_cloud.paint_uniform_color(segment_color)

        # Append the segment cloud to the geometries list
        geometries.append(segment_cloud)

    # Visualize all segments
    o3d.visualization.draw_geometries(geometries, point_show_normal=True)